In [ ]:
import json
import math
import numpy as np
from collections import defaultdict
import random
from google.colab import files

print("Upload T10I4D100K.txt")
uploaded = files.upload()

def encode_dataset(filepath='T10I4D100K.txt'):
    """
    Encode string items into integer IDs.
    Returns:
      - encoded_data: list of tuples of ints
      - item_to_id: dict item string -> int
      - id_to_item: dict int -> item string
    """
    item_to_id = {}
    id_to_item = {}
    encoded_data = []

    with open(filepath, 'r') as f:
        for line in f:
            items = line.strip().split()
            encoded_line = []
            for item in items:
                if item not in item_to_id:
                    new_id = len(item_to_id)
                    item_to_id[item] = new_id
                    id_to_item[new_id] = item
                encoded_line.append(item_to_id[item])
            encoded_data.append(tuple(encoded_line))
    return encoded_data, item_to_id, id_to_item

class ItemDataSet():
    def __init__(self, encoded_data=None, item_to_id=None, id_to_item=None):
        self.record = encoded_data if encoded_data is not None else []
        self.item_to_id = item_to_id if item_to_id is not None else {}
        self.id_to_item = id_to_item if id_to_item is not None else {}

    def add_line(self, line):
        self.record.append(line)

    def get_line(self, index):
        return self.record[index]

    def get_line_num(self):
        return len(self.record)

    def __getitem__(self, index):
        return self.get_line(index)

    def init_candidate(self):
        unique_items = set()
        for transaction in self.record:
            unique_items.update(transaction)
        return [(item,) for item in sorted(unique_items)]

def sampleClients(num_participants, duplicate, orig_traj_num):
    clients = []
    for _ in range(duplicate):
        clients.extend(list(range(orig_traj_num)))
    if num_participants <= len(clients):
        return np.random.choice(clients, num_participants, replace=False)
    else:
        return np.random.choice(clients, num_participants, replace=True)

def printRound(i):
    print('==================================')
    print(f'             Round {i}             ')
    print('==================================')

class Args:
    def __init__(self, k, epsilon):
        self.k = k
        self.epsilon = epsilon
        self.duplicate = 1
        self.num_participants = 3000000
        self.verbose = False
        self.pattern_type = 'itemset'
        self.mode = 'ffpa'
        self.orig_k = self.k
        self.round = 0
        self.num_candidate = 30
        self.max_rounds = 1

def groundTruth(dataset, args):
    k = max(1, int(args.k * dataset.get_line_num()))  # absolute count
    traj_num = dataset.get_line_num()
    freq_itemsets = dict()

    round_num = 1
    candidates = dataset.init_candidate()

    while round_num <= args.max_rounds:
        printRound(round_num)
        print(f"{round_num}-itemsets: {len(candidates)} candidates")
        if len(candidates) == 0:
            print("No more candidates, stop.")
            break

        support_count = defaultdict(int)
        participants = sampleClients(args.num_participants, args.duplicate, traj_num)

        if round_num == 1:
            candidate_items = set(c[0] for c in candidates)
            for idx in participants:
                transaction = dataset.get_line(idx)
                for item in transaction:
                    if item in candidate_items:
                        support_count[(item,)] += 1
        else:
            for idx in participants:
                transaction = set(dataset.get_line(idx))
                for candi in candidates:
                    if set(candi).issubset(transaction):
                        support_count[candi] += 1

        freq_candidates = [c for c in support_count if support_count[c] >= k]
        for c in freq_candidates:
            freq_itemsets[c] = support_count[c]

        print(f"{round_num}-itemsets frequent: {len(freq_candidates)}")

        if len(freq_candidates) == 0:
            break
        if round_num == args.max_rounds:
            print(f"Reached max rounds ({args.max_rounds}), stop.")
            break

        new_candidates = []
        freq_candidates_sorted = sorted(freq_candidates)
        for i in range(len(freq_candidates_sorted)):
            for j in range(i+1, len(freq_candidates_sorted)):
                c1 = freq_candidates_sorted[i]
                c2 = freq_candidates_sorted[j]
                if c1[:-1] == c2[:-1]:
                    candidate = tuple(sorted(set(c1) | set(c2)))
                    if candidate not in new_candidates:
                        new_candidates.append(candidate)
        candidates = new_candidates
        round_num += 1

    print(f"Total rounds performed: {round_num-1}")
    return freq_itemsets

class FfpaHandler:
    def __init__(self, args, dataset):
        self.args = args
        self.dataset = dataset
        self.traj_num = dataset.get_line_num()
        self.k = max(1, int(args.k * self.traj_num))  # absolute count
        self.candidate_pool = set(dataset.init_candidate())
        self.accepted = set()
        self.round = 0

    def randomize_bit(self, bit, epsilon):
        p = math.exp(epsilon) / (math.exp(epsilon) + 1)
        if random.random() < p:
            return bit
        else:
            return 1 - bit

    def run(self):
        round_num = 1
        candidates = list(self.candidate_pool)
        accepted_candidates = set()

        while round_num <= self.args.max_rounds and len(candidates) > 0:
            printRound(round_num)
            print(f"{round_num}-candidates: {len(candidates)}")
            support_count = defaultdict(float)

            participants = sampleClients(self.args.num_participants, self.args.duplicate, self.traj_num)

            for idx in participants:
                transaction = set(self.dataset.get_line(idx))
                asked_candidates = random.sample(candidates, min(len(candidates), self.args.num_candidate))
                for candi in asked_candidates:
                    bit = 1 if set(candi).issubset(transaction) else 0
                    noisy_bit = self.randomize_bit(bit, self.args.epsilon)
                    support_count[candi] += noisy_bit

            p = math.exp(self.args.epsilon) / (math.exp(self.args.epsilon) + 1)
            q = 1 - p
            est_supports = dict()
            for candi in candidates:
                noisy = support_count[candi]
                est = (noisy - q * self.args.num_participants) / (p - q)
                est_supports[candi] = est

            accepted = [c for c, est in est_supports.items() if est >= self.k]
            print(f"Accepted candidates: {len(accepted)}")

            accepted_candidates.update(accepted)

            new_candidates = []
            accepted_sorted = sorted(accepted)
            for i in range(len(accepted_sorted)):
                for j in range(i+1, len(accepted_sorted)):
                    c1 = accepted_sorted[i]
                    c2 = accepted_sorted[j]
                    if c1[:-1] == c2[:-1]:
                        candidate = tuple(sorted(set(c1) | set(c2)))
                        if candidate not in accepted_candidates and candidate not in new_candidates:
                            new_candidates.append(candidate)

            candidates = new_candidates
            round_num += 1

        self.round = round_num - 1
        print(f"FFPA total rounds: {self.round}")
        return accepted_candidates

def calc_metrics(result, truth):
    tp = sum(1 for frag in result if frag in truth)
    fp = len(result) - tp
    fn = len(truth) - tp
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print(f"TP={tp}, FP={fp}, FN={fn}")
    return precision, recall, f1

def main():
    dataset, item_to_id, id_to_item = encode_dataset('T10I4D100K.txt')
    dataset = ItemDataSet(dataset, item_to_id, id_to_item)
    epsilons = [1.0, 3.0, 5.0, 7.0, 9.0]
    min_sups = [0.003,0.005,0.008, 0.01,0.03]

    results = []

    for epsilon in epsilons:
        for min_sup in min_sups:
            print("="*35)
            print(f"Testing with epsilon={epsilon} and k proportion={min_sup} (k={int(min_sup*dataset.get_line_num())})")
            print("="*35)

            args = Args(k=min_sup, epsilon=epsilon)
            print("\n--- Ground Truth Mining ---")
            gt_itemsets = groundTruth(dataset, args)
            print(f"Ground truth frequent itemsets: {len(gt_itemsets)}")

            print("\n--- FFPA Mining ---")
            handler = FfpaHandler(args, dataset)
            ffpa_res = handler.run()
            print(f"FFPA found itemsets: {len(ffpa_res)}")

            print("\n--- Evaluation Metrics ---")
            precision, recall, f1 = calc_metrics(list(ffpa_res), list(gt_itemsets.keys()))

            results.append({
                "dataset_name": "T10I4D100K",
                "epsilon": epsilon,
                "min_sup": min_sup,
                "f1_score": f1
            })

    with open('ffpa_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    print("Results saved to ffpa_results.json")

if __name__ == '__main__':
    main()
